In [1]:
import requests
import pandas as pd
import json
import numpy as np
from datetime import datetime, timezone
import snowflake.connector
import os
from dotenv import load_dotenv

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
load_dotenv()

# Configuration des APIs
AQICN_API_KEY = os.getenv('AQICN_API_KEY')
OPENWEATHER_API_KEY = os.getenv('OPENWEATHER_API_KEY')


#SNOWFLAKE
ACOUNT_SNOWFLAKE = os.getenv('ACOUNT_SNOWFLAKE')

conn = snowflake.connector.connect(
    user="LOUK",
    password="Snowflake_08230!",
    account=ACOUNT_SNOWFLAKE,
    warehouse="COMPUTE_WH",
    database="GOOD_AIR",
    schema="RAW"
)
cur = conn.cursor()


# Villes françaises avec coordonnées
CITIES = [
    ("Paris", 48.8566, 2.3522),
    ("Marseille", 43.2965, 5.3698),
    ("Lyon", 45.7640, 4.8357),
    ("Toulouse", 43.6045, 1.4440),
    ("Nice", 43.7102, 7.2620),
    ("Nantes", 47.2184, -1.5536),
    ("Montpellier", 43.6109, 3.8772),
    ("Strasbourg", 48.5734, 7.7521),
    ("Bordeaux", 44.8378, -0.5792),
    ("Lille", 50.6292, 3.0573),
    ("Rennes", 48.1173, -1.6778),
    ("Reims", 49.2583, 4.0317),
    ("Saint-Étienne", 45.4397, 4.3872),
    ("Le Havre", 49.4944, 0.1079),
    ("Toulon", 43.1242, 5.9280),
    ("Grenoble", 45.1885, 5.7245),
    ("Dijon", 47.3220, 5.0415),
    ("Angers", 47.4784, -0.5632),
    ("Nîmes", 43.8367, 4.3601),
    ("Villeurbanne", 45.7719, 4.8902)
]

OPENWEATHER

In [ ]:
# ⚙️ Paramètres
units = "metric"
lang = "fr"

#Array pour recevoir les reponse, un element pour une ville
data_cities = []

#boucle sur les ville selectionnées
for city in CITIES :
    name_city = city[0] 
    lat = city[1]
    lon = city[2]

    # 🔗 URL One Call 2.5
    url = (
        f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={OPENWEATHER_API_KEY}&units=metric&lang=fr"
    )

    response = requests.get(url)

    try:
        response.raise_for_status()  
        data = response.json()
        data_cities.append(data)
        
    except requests.exceptions.HTTPError as http_err:
        print(f"❌ Erreur HTTP : {response.status_code} - {http_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"⚠️ Erreur de requête : {req_err}")
    except ValueError:
        print("⚠️ Réponse reçue mais le JSON est invalide.")
print("✅ Requête réussie :")

✅ Requête réussie : {'coord': {'lon': 2.3494, 'lat': 48.8558}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'couvert', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 16.47, 'feels_like': 16.44, 'temp_min': 15.2, 'temp_max': 17.1, 'pressure': 1009, 'humidity': 87, 'sea_level': 1009, 'grnd_level': 999}, 'visibility': 10000, 'wind': {'speed': 6.17, 'deg': 180}, 'clouds': {'all': 100}, 'dt': 1756715364, 'sys': {'type': 2, 'id': 2012208, 'country': 'FR', 'sunrise': 1756703255, 'sunset': 1756751644}, 'timezone': 7200, 'id': 2978048, 'name': 'Quartier Saint-Merri', 'cod': 200}
✅ Requête réussie : {'coord': {'lon': 5.37, 'lat': 43.2962}, 'weather': [{'id': 502, 'main': 'Rain', 'description': 'forte pluie', 'icon': '10d'}], 'base': 'stations', 'main': {'temp': 19.27, 'feels_like': 19.71, 'temp_min': 19.27, 'temp_max': 19.99, 'pressure': 1010, 'humidity': 94, 'sea_level': 1010, 'grnd_level': 1003}, 'visibility': 10000, 'wind': {'speed': 7.72, 'deg': 110}, 'rain': {'1h': 5.62},

In [5]:
#converstion en data frame
data_cities

[{'coord': {'lon': 2.3494, 'lat': 48.8558},
  'weather': [{'id': 804,
    'main': 'Clouds',
    'description': 'couvert',
    'icon': '04d'}],
  'base': 'stations',
  'main': {'temp': 16.47,
   'feels_like': 16.44,
   'temp_min': 15.2,
   'temp_max': 17.1,
   'pressure': 1009,
   'humidity': 87,
   'sea_level': 1009,
   'grnd_level': 999},
  'visibility': 10000,
  'wind': {'speed': 6.17, 'deg': 180},
  'clouds': {'all': 100},
  'dt': 1756715364,
  'sys': {'type': 2,
   'id': 2012208,
   'country': 'FR',
   'sunrise': 1756703255,
   'sunset': 1756751644},
  'timezone': 7200,
  'id': 2978048,
  'name': 'Quartier Saint-Merri',
  'cod': 200},
 {'coord': {'lon': 5.37, 'lat': 43.2962},
  'weather': [{'id': 502,
    'main': 'Rain',
    'description': 'forte pluie',
    'icon': '10d'}],
  'base': 'stations',
  'main': {'temp': 19.27,
   'feels_like': 19.71,
   'temp_min': 19.27,
   'temp_max': 19.99,
   'pressure': 1010,
   'humidity': 94,
   'sea_level': 1010,
   'grnd_level': 1003},
  'visib

ENVOIS DANS SNOWFLAKE

In [ ]:
#convertion de al data en json
json_data = json.dumps(data_cities)

try:
    # Insertion dans la table
    cur.execute("""
        INSERT INTO weather_api(raw_json) 
        SELECT PARSE_JSON(%s)
    """, (json_data,))
    
    conn.commit()
    print("✅ Insertion réussie")

except Exception as e:
    print("❌ Erreur lors de l'insertion :", e)
    conn.rollback()

finally:
    cur.close()


✅ Insertion réussie


AQICN

In [3]:
#Array pour recevoir les reponse, un element pour une ville
data_cities = []

#boucle sur les ville selectionnées
for city in CITIES :
    name_city = city[0] 
    lat = city[1]
    lon = city[2]

    # 🔗 URL One Call 2.5
    url = (
        f"https://api.waqi.info/feed/{name_city}/?token={AQICN_API_KEY}"
    )

    response = requests.get(url)

    try:
        response.raise_for_status()  
        data = response.json()
        data_cities.append(data)
        print("✅ Requête réussie :", data)
    except requests.exceptions.HTTPError as http_err:
        print(f"❌ Erreur HTTP : {response.status_code} - {http_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"⚠️ Erreur de requête : {req_err}")
    except ValueError:
        print("⚠️ Réponse reçue mais le JSON est invalide.")

✅ Requête réussie : {'status': 'ok', 'data': {'aqi': 11, 'idx': 5722, 'attributions': [{'url': 'https://www.airparif.asso.fr/', 'name': "AirParif - Association de surveillance de la qualité de l'air en Île-de-France", 'logo': 'Paris-Air-Parif.png'}, {'url': 'http://www.eea.europa.eu/themes/air/', 'name': 'European Environment Agency', 'logo': 'Europe-EEA.png'}, {'url': 'https://waqi.info/', 'name': 'World Air Quality Index Project'}], 'city': {'geo': [48.856614, 2.3522219], 'name': 'Paris', 'url': 'https://aqicn.org/city/paris', 'location': ''}, 'dominentpol': 'pm10', 'iaqi': {'co': {'v': 0.1}, 'h': {'v': 89.8}, 'no2': {'v': 24.9}, 'o3': {'v': 10.7}, 'p': {'v': 1011.7}, 'pm10': {'v': 11}, 'pm25': {'v': 27}, 'so2': {'v': 0.6}, 't': {'v': 13.1}, 'w': {'v': 0.5}}, 'time': {'s': '2025-09-02 05:00:00', 'tz': '+02:00', 'v': 1756789200, 'iso': '2025-09-02T05:00:00+02:00'}, 'forecast': {'daily': {'o3': [{'avg': 8, 'day': '2025-09-02', 'max': 11, 'min': 4}, {'avg': 10, 'day': '2025-09-03', 'max

In [4]:
data_cities

[{'status': 'ok',
  'data': {'aqi': 11,
   'idx': 5722,
   'attributions': [{'url': 'https://www.airparif.asso.fr/',
     'name': "AirParif - Association de surveillance de la qualité de l'air en Île-de-France",
     'logo': 'Paris-Air-Parif.png'},
    {'url': 'http://www.eea.europa.eu/themes/air/',
     'name': 'European Environment Agency',
     'logo': 'Europe-EEA.png'},
    {'url': 'https://waqi.info/', 'name': 'World Air Quality Index Project'}],
   'city': {'geo': [48.856614, 2.3522219],
    'name': 'Paris',
    'url': 'https://aqicn.org/city/paris',
    'location': ''},
   'dominentpol': 'pm10',
   'iaqi': {'co': {'v': 0.1},
    'h': {'v': 89.8},
    'no2': {'v': 24.9},
    'o3': {'v': 10.7},
    'p': {'v': 1011.7},
    'pm10': {'v': 11},
    'pm25': {'v': 27},
    'so2': {'v': 0.6},
    't': {'v': 13.1},
    'w': {'v': 0.5}},
   'time': {'s': '2025-09-02 05:00:00',
    'tz': '+02:00',
    'v': 1756789200,
    'iso': '2025-09-02T05:00:00+02:00'},
   'forecast': {'daily': {'o3': 

In [ ]:
#convertion de al data en json
json_data = json.dumps(data_cities)

try:
    # Insertion dans la table
    cur.execute("""
        INSERT INTO aqicn_api(raw_json) 
        SELECT PARSE_JSON(%s)
    """, (json_data,))
    
    conn.commit()
    print("✅ Insertion réussie")

except Exception as e:
    print("❌ Erreur lors de l'insertion :", e)
    conn.rollback()

finally:
    cur.close()


✅ Insertion réussie
